In [47]:
!pip install googlemaps
!pip install odfpy

In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import googlemaps
from datetime import datetime


if 'acidentes.csv' not in os.listdir():
  !wget https://raw.githubusercontent.com/namespaiva/acidentes/main/dados/acidentes.csv
df = pd.read_csv('acidentes.csv')

gmaps = googlemaps.Client(key='chave')

# Código residual. Tô com dó de apagar.
# for ano in range(2018, 2020):
#   if f'Acidentes {ano}.ods' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/acidentes/main/dados/Acidentes%20{ano}.ods

# for ano in range(2020, 2025):
#   if f'Acidentes {ano}.xls' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/acidentes/main/dados/Acidentes%20{ano}.xls


# dados = pd.read_excel('Acidentes 2018.ods', engine='odf')
# dados = pd.concat([dados, pd.read_excel('Acidentes 2019.ods', engine='odf')])

# dados = pd.read_excel('Acidentes 2020.xls')

# for ano in range(2021, 2025):
#   dados = pd.concat([dados, pd.read_excel(f'Acidentes {ano}.xls')])

# df = dados.sort_values(by='DATA')

# df['CRUZAMENTO'] = df['CRUZAMENTO'].astype(str) + df['CRUZAMENTO.1'].astype(str)
# df['LOGRADOURO'] = df['LOGRADOURO'].astype(str) + df['LOGRADOURO1'].astype(str)
# df = df.drop(columns=['LOGRADOURO1', 'CRUZAMENTO.1', 'Unnamed: 0', 2])
# df.rename(columns={'Nº': 'NUMERO'}, inplace=True)
# df.to_csv('acidentes.csv', index=False)

def remove_nans(s):
    s = str(s)
    if s.startswith("nan"):
        return s[3:]  # Skip the first 3 characters if they are "nan"
    elif s.endswith("nan"):
        return s[:-3]  # Skip the last 3 characters if they are "nan"
    else:
        return s

df['LOGRADOURO'] = df['LOGRADOURO'].apply(remove_nans)
df['CRUZAMENTO'] = df['CRUZAMENTO'].apply(remove_nans)

df['DATA'] = pd.to_datetime(df['DATA'])
df['NUMERO'] = df['NUMERO'].astype(str)
df['lat'] = ''
df['lng'] = ''
df['types'] = ''
df['bairro'] = ''
df.rename(columns={'DATA': 'data', 'HORA': 'hora', 'TEMPO': 'tempo',
                   'TIPO_ACIDENTE': 'tipo_acidente', 'GRAVIDADE': 'gravidade',
                   'LOGRADOURO': 'logradouro', 'NUMERO': 'numero', 'CRUZAMENTO': 'cruzamento'}, inplace=True)

In [85]:
gravidades = list(df['gravidade'].unique())

for index, row in df.iterrows():
  print(row['gravidade'])
  if trim(row['gravidade']) == gravidades[4]:
    df.at[index, 'gravidade'] = 'nan'
    print('fodase')
  if row['gravidade'] == ' ':
    df.at[index, 'gravidade'] = 'nan'

df['gravidade'].unique()


S/ LESÃO


NameError: name 'trim' is not defined

In [57]:
gravidades = list(df['gravidade'].unique())
df['temp'] = df['cruzamento']

for index, row in df.iterrows():
  for gravidade in gravidades:
    if str(gravidade) in row['cruzamento']:
      df.at[index, 'gravidade'] = str(gravidade)
      df.at[index, 'temp'] = row['temp'].replace(str(gravidade), '')

df['cruzamento'] = df['temp']
df.drop(columns=['temp'], axis=1, inplace=True)

In [63]:
df[df['gravidade'] == ' ']

,data,hora,tempo,tipo_acidente,gravidade,logradouro,numero,cruzamento,lat,lng,types,bairro
0,2020-01-01,02:53:00,BOM,COLISÃO,,Rua Arabutan,0,RuaProfessorPirajádaSilva,-23.976606,-46.306146,[intersection],Aparecida
1,2020-01-01,03:42:00,BOM,COLISÃO,,Rua Bittencourt,0,AvenidaSenadorFeijó,-23.938324,-46.325695,[intersection],Vila Nova
4,2020-01-02,23:16:00,CHUVA,COLISÃO,,Rua Ministro Xavier de Toledo,0,RuaViscondedeFaria,-23.961088,-46.3406,[intersection],Campo Grande
6,2020-01-02,20:22:00,CHUVA,COLISÃO,,Rua André Vidal de Negreiros,0,RuaProfessorDoutorOlavodePaulaBorges,-23.984042,-46.303007,[intersection],Ponta da Praia
9,2020-01-02,18:17:00,CHUVA,COLISÃO,,Avenida General Francisco Glycerio,0,AvenidaSenadorPinheiroMachado,-23.964153,-46.345613,[intersection],José Menino
...,...,...,...,...,...,...,...,...,...,...,...,...
9474,2024-05-31,14:02:00,BOM,COLISÃO,,Praça Francisco Martins dos Santos,0,TúnelRubensFerreiraMartins,-23.937947,-46.333497,[route],Santos
9475,2024-05-31,07:58:00,BOM,TOMBAMENTO,,Avenida Conselheiro Rodrigues Alves,0,RuaRodrigoSilva,-23.95681,-46.309888,[intersection],Santos
9476,2024-05-31,08:36:00,BOM,COLISÃO,,Avenida Hugo Maia,0,AvenidaJovinodeMello,-23.936643,-46.380768,[intersection],Bom Retiro
9478,2024-05-31,07:17:00,BOM,COLISÃO,,Avenida Conselheiro Nébias,0,RuaLuizGama,-23.954716,-46.322633,[intersection],Encruzilhada


In [59]:
gravidades[0]

'S/ LESÃO'

In [50]:
for index, row in df.iterrows():
  if row['cruzamento'] == 'nan':
    local = row['numero'] + ' ' + row['logradouro']
  else:
    local = row['logradouro'] + ' ' + row['cruzamento']

  #print(local)
  geocode_result = gmaps.geocode(f'{local}, Santos, SP, Brazil')

  if row['cruzamento'] == 'nan':
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][2]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'
  else:
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][1]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'

  df.at[index, 'types'] = geocode_result[0]['types']
  df.at[index, 'lat'] = geocode_result[0]['geometry']['location']['lat']
  df.at[index, 'lng'] = geocode_result[0]['geometry']['location']['lng']

In [60]:
import folium
from folium.plugins import HeatMap

m = folium.Map(location=(-23.959, -46.332), zoom_start=12)

HeatMap(data=df[['lat', 'lng']], radius=15).add_to(m)

m

In [62]:
df.to_csv('acidentesupdate.csv', index=False)
df['gravidade'].unique()

array([' ', 'S/ LESÃO', 'C/ VÍTIMAS LEVES', 'C/ VÍTIMAS GRAVES',
       'C/ VÍTIMAS FATAIS'], dtype=object)